### Example code for finding local optima for mixed strategies

In this example, we are looking at two players who can either play:
 - classical long 
 $$
 \begin{pmatrix}
 1 & 0 \\
 0 & 1
 \end{pmatrix}
 $$
 - quantum long
  $$
 \begin{pmatrix}
 i & 0 \\
 0 & -1
 \end{pmatrix}
 $$
 - quantum short
  $$
 \begin{pmatrix}
 0 & -i \\
 -i & 0
 \end{pmatrix}
 $$
 - classical short
  $$
 \begin{pmatrix}
 0 & 1 \\
 -1 & 0
 \end{pmatrix}
 $$

 Each player will play each strategy with probability p0, p1, p2, or p3 respectively. We assume both players are using the same set of probabilities.

 To define the game they are playing, we need to determine the payoff to each player based on the measurement outcome. This function will input a dataframe of shot outcomes for each player, and output a dataframe of the payoff for each player for that shot. Here is a simple example:

In [1]:
import pandas as pd

def two_player_prisoners_dilemma_payoff(results_df: pd.DataFrame):
    # Define payoff matrix
    payoff_matrix = [[(3,3), (5,0)],
                     [(0,5), (1,1)]]

    # Create payoff Dataframe
    payoff_df = results_df.copy()

    # Assign player names to the variables p1 and p2
    p1 = results_df.columns[0]
    p2 = results_df.columns[1]

    for shot in results_df.iterrows():
        # Determine the circuit result for each player
        p1_result = int(shot[1][p1])
        p2_result = int(shot[1][p2])

        # Look up payoffs in the payoff matrix
        payoffs = payoff_matrix[p1_result][p2_result]

        # Assing the resulting payoff to the proper row in the Dataframe
        payoff_df.at[shot[0], p1] = payoffs[0]
        payoff_df.at[shot[0], p2] = payoffs[1]

    return payoff_df

Here is a demonstration of how the payoff function works. We begin with two players who played the game 3 times, with the resulting measurements |1>, |0>, |1> and |0>, |0>, |1> respectively. Here would be their payoffs for each round of the simple prisoners dilemma game:

In [2]:
test_df = pd.DataFrame({'player_1': [1, 0, 1],
 'player_2': [0, 0, 1]})

two_player_prisoners_dilemma_payoff(test_df)

,player_1,player_2
0,0,5
1,3,3
2,1,1


Next, we create a minimizer using the defined payoff function. The minimizer takes optional arguments that can be used to modify the number of players, their pure strategy options, the number of shots per iteration, and the circuit execution method.

In [3]:
from Quantum_Games import minimize_mixed_strategies

minimizer = minimize_mixed_strategies(two_player_prisoners_dilemma_payoff)

We then call the minimizer.minimize() method. By default this will use the Scipy.optimize Nelder-Mead implementation, however we can use any optional arguments from the Scipy optimizer - https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

To demonstrate let's try with the COBYLA optimizer:

In [5]:
result = minimizer.minimize(method='COBYLA')

The result gives us the full optimization result information. Recall our definition of p1, p2, p3 and p4 from above. Let the optimized parameter vector $$x = [x_1, x_2, x_3]$$, then the quantum mixed strategy is p1 = x1, p2=x2, p3=x3, p4= 1-x1-x2-x3.

In [7]:
print('probability of classical long:', result.x[0])
print('probability of quantum long:', result.x[1])
print('probability of quantum short:', result.x[2])
print('probability of classical short:', 1-sum(result.x))

probability of classical long: 0.31027461063121226
probability of quantum long: 0.21974676836376072
probability of quantum short: 0.381897278629054
probability of classical short: 0.08808134237597298
